In [7]:
import os
import numpy as np
import pandas as pd
from scipy.io import wavfile
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from scipy.signal import spectrogram
from collections import Counter
import random





In [8]:
#4.seçenek
metadata_path = r"C:\Users\İREM\Desktop\archive\UrbanSound8K.csv"
metadata = pd.read_csv(metadata_path)

print("Toplam ses dosyası sayısı:", len(metadata))
print(metadata.head())


Toplam ses dosyası sayısı: 8732
      slice_file_name    fsID  start        end  salience  fold  classID  \
0    100032-3-0-0.wav  100032    0.0   0.317551         1     5        3   
1  100263-2-0-117.wav  100263   58.5  62.500000         1     5        2   
2  100263-2-0-121.wav  100263   60.5  64.500000         1     5        2   
3  100263-2-0-126.wav  100263   63.0  67.000000         1     5        2   
4  100263-2-0-137.wav  100263   68.5  72.500000         1     5        2   

              class  
0          dog_bark  
1  children_playing  
2  children_playing  
3  children_playing  
4  children_playing  


In [9]:
def extract_features(file_path):
    try:
        # WAV dosyasını oku
        sample_rate, data = wavfile.read(file_path)

        # Eğer stereo ise mono’ya çevir
        if data.ndim > 1:
            data = np.mean(data, axis=1)
        
        # Tüm sesleri aynı uzunluğa getirmek için
        target_length = 160000
        if len(data) < target_length:
            padding = np.zeros(target_length - len(data))
            data = np.concatenate((data, padding))
        else:
            data = data[:target_length]

        # Basit istatistiksel özellikler çıkarma
        features = {
            "mean": np.mean(data),
            "std": np.std(data),
            "max": np.max(data),
            "min": np.min(data),
            "median": np.median(data)
        }
        return features
    except Exception as e:
        print(f"Hata ({file_path}):", e)
        return None


In [10]:
features_list = []

for i, row in metadata.head(200).iterrows():  # ilk 200 sesle test
    fold = row["fold"]
    file_name = row["slice_file_name"]
    class_id = row["classID"]
    
    # Ses dosyasının tam yoluna ulaşması
    file_path = fr"C:\Users\İREM\Desktop\archive\fold{fold}\{file_name}"
    
    if os.path.exists(file_path):
        features = extract_features(file_path)
        if features:
            features["classID"] = class_id
            features_list.append(features)
    else:
        print("❌ Dosya bulunamadı:", file_path)

# Verileri tablo haline getirme
features_df = pd.DataFrame(features_list)
print("✅ Toplam çıkarılan özellik sayısı:", len(features_df))
print(features_df.head())


✅ Toplam çıkarılan özellik sayısı: 200
       mean          std      max      min  median  classID
0 -0.015159  1593.801285  32767.0 -31682.5     0.0        3
1 -1.994638   137.365322    957.0   -840.5    -1.5        2
2 -3.192325   100.612211    492.5   -528.0    -3.5        2
3 -3.049853   163.943474   1066.0  -1077.0    -4.0        2
4 -2.641044   106.376505    471.5   -490.0    -1.5        2


In [11]:
# Özellikleri ve etiketleri ayırma
X = features_df.drop(columns=["classID"])
y = features_df["classID"]

# Eğitim ve test verisine bölme
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Modeli oluştur ve eğitme
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Test setinde tahmin yapma
y_pred = model.predict(X_test)

# Sonuçları değerlendirme
print("🎯 Doğruluk Oranı:", accuracy_score(y_test, y_pred))
print("\n📊 Sınıflandırma Raporu:\n", classification_report(y_test, y_pred))
print("\n🌀 Karışıklık Matrisi:\n", confusion_matrix(y_test, y_pred))


🎯 Doğruluk Oranı: 0.925

📊 Sınıflandırma Raporu:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       1.00      1.00      1.00         1
           2       0.33      0.50      0.40         2
           3       0.83      0.71      0.77         7
           5       1.00      1.00      1.00         9
           7       1.00      1.00      1.00         4
           8       1.00      1.00      1.00         5
           9       1.00      1.00      1.00         1

    accuracy                           0.93        40
   macro avg       0.90      0.90      0.90        40
weighted avg       0.94      0.93      0.93        40


🌀 Karışıklık Matrisi:
 [[11  0  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  0  0]
 [ 0  0  1  1  0  0  0  0]
 [ 0  0  2  5  0  0  0  0]
 [ 0  0  0  0  9  0  0  0]
 [ 0  0  0  0  0  4  0  0]
 [ 0  0  0  0  0  0  5  0]
 [ 0  0  0  0  0  0  0  1]]


In [12]:
#6.seçenek
#Ses dosyasından öznitelik çıkarımı

def extract_features(file_path):
    try:
        sr, data = wavfile.read(file_path)
        if data.ndim > 1:
            data = np.mean(data, axis=1)
        target_length = 160000
        if len(data) < target_length:
            data = np.pad(data, (0, target_length - len(data)))
        else:
            data = data[:target_length]
        return [np.mean(data), np.std(data), np.max(data), np.min(data), np.median(data)]
    except:
        return None


In [24]:
#  Veriyi oku ve özellikleri oluştur
features, labels = [], []
for i, row in metadata.head(200).iterrows():
    file_path = fr"C:\Users\İREM\Desktop\archive\fold{row['fold']}\{row['slice_file_name']}"
    if os.path.exists(file_path):
        f = extract_features(file_path)
        if f:
            features.append(f)
            labels.append(row["classID"])

X = np.array(features)
y = np.array(labels)
print("✅ Özellik vektörleri boyutu:", X.shape)

✅ Özellik vektörleri boyutu: (200, 5)


In [25]:
#  Eğitim/Test bölünmesi

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [26]:
#  Basit Karar Ağacı Fonksiyonları

def gini_index(y):
    """Gini impurity hesaplar."""
    classes = np.unique(y)
    gini = 1.0
    for c in classes:
        p = np.sum(y == c) / len(y)
        gini -= p ** 2
    return gini

def split_data(X, y, feature_index, threshold):
    """Veriyi belirli özelliğe göre ikiye ayırır."""
    left_idx = X[:, feature_index] <= threshold
    right_idx = X[:, feature_index] > threshold
    return X[left_idx], y[left_idx], X[right_idx], y[right_idx]

def best_split(X, y):
    """En iyi bölmeyi bulur."""
    best_feature, best_threshold, best_gini = None, None, 1.0
    n_features = X.shape[1]
    for feature_index in range(n_features):
        thresholds = np.unique(X[:, feature_index])
        for t in thresholds:
            _, y_left, _, y_right = split_data(X, y, feature_index, t)
            if len(y_left) == 0 or len(y_right) == 0:
                continue
            gini = (len(y_left)/len(y))*gini_index(y_left) + (len(y_right)/len(y))*gini_index(y_right)
            if gini < best_gini:
                best_gini = gini
                best_feature = feature_index
                best_threshold = t
    return best_feature, best_threshold

class DecisionTree:
    """Basit karar ağacı sınıfı."""
    def __init__(self, max_depth=3):
        self.max_depth = max_depth
        self.feature_index = None
        self.threshold = None
        self.left = None
        self.right = None
        self.label = None

    def fit(self, X, y, depth=0):
        if len(np.unique(y)) == 1:
            self.label = y[0]
            return
        if depth >= self.max_depth or len(y) < 2:
            self.label = Counter(y).most_common(1)[0][0]
            return

        feature, threshold = best_split(X, y)
        if feature is None:
            self.label = Counter(y).most_common(1)[0][0]
            return

        self.feature_index = feature
        self.threshold = threshold
        X_left, y_left, X_right, y_right = split_data(X, y, feature, threshold)

        self.left = DecisionTree(self.max_depth)
        self.right = DecisionTree(self.max_depth)
        self.left.fit(X_left, y_left, depth + 1)
        self.right.fit(X_right, y_right, depth + 1)

    def predict_one(self, x):
        if self.label is not None:
            return self.label
        if x[self.feature_index] <= self.threshold:
            return self.left.predict_one(x)
        else:
            return self.right.predict_one(x)

In [6]:
#  Random Forest 

class RandomForest:
    def __init__(self, n_trees=5, max_depth=3, sample_ratio=0.8):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.sample_ratio = sample_ratio
        self.trees = []

    def bootstrap_sample(self, X, y):
        n_samples = int(len(X) * self.sample_ratio)
        indices = np.random.choice(len(X), n_samples, replace=True)
        return X[indices], y[indices]

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            X_s, y_s = self.bootstrap_sample(X, y)
            tree = DecisionTree(max_depth=self.max_depth)
            tree.fit(X_s, y_s)
            self.trees.append(tree)

    def predict(self, X):
        all_preds = np.array([[tree.predict_one(x) for x in X] for tree in self.trees])
        final_preds = []
        for col in all_preds.T:
            final_preds.append(Counter(col).most_common(1)[0][0])
        return np.array(final_preds)


In [27]:
#  Modeli eğit ve değerlendir

model = RandomForest(n_trees=5, max_depth=4)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)


In [28]:
# 8️⃣ Metrikler (sklearn’suz)
# -------------------------------------------------------------------
num_classes = max(np.max(y_test), np.max(y_pred)) + 1
cm = np.zeros((num_classes, num_classes), dtype=int)

for true, pred in zip(y_test, y_pred):
    cm[true, pred] += 1

# Accuracy
accuracy = np.sum(y_test == y_pred) / len(y_test)

# Precision ve Recall (macro)
precision_list = []
recall_list = []

for i in range(num_classes):
    tp = cm[i, i]
    fp = np.sum(cm[:, i]) - tp
    fn = np.sum(cm[i, :]) - tp
    
    precision = tp / (tp + fp) if (tp + fp) != 0 else 0
    recall = tp / (tp + fn) if (tp + fn) != 0 else 0
    
    precision_list.append(precision)
    recall_list.append(recall)

precision_macro = np.mean(precision_list)
recall_macro = np.mean(recall_list)

print("\n🎯 Doğruluk (Accuracy):", round(accuracy, 3))
print("📏 Kesinlik (Precision):", round(precision_macro, 3))
print("📈 Duyarlılık (Recall):", round(recall_macro, 3))
print("\n🌀 Karışıklık Matrisi:\n", cm)


🎯 Doğruluk (Accuracy): 0.825
📏 Kesinlik (Precision): 0.494
📈 Duyarlılık (Recall): 0.518

🌀 Karışıklık Matrisi:
 [[11  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  0  0  0  0]
 [ 0  0  2  0  0  0  0  0  0  0]
 [ 2  0  2  3  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  9  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  3  0  0]
 [ 0  0  0  0  0  0  0  0  5  0]
 [ 0  0  1  0  0  0  0  0  0  0]]
